# Multivariate Time Series
Multivariate time series adalah rangkaian data di mana setiap titik data terdiri dari beberapa variabel yang diamati atau diukur secara bersamaan dalam interval waktu yang berurutan. Berbeda dengan univariate time series, kita tidak hanya melihat satu variabel pada satu waktu tertentu, tetapi sejumlah variabel yang diamati secara bersamaan. Ini memungkinkan kita untuk memahami bagaimana variabel-variabel tersebut berinteraksi satu sama lain seiring waktu.

Dengan kata lain, multivariate time series adalah cara untuk menganalisis dan memodelkan hubungan kompleks antara berbagai variabel dalam rentang waktu tertentu. Ini memberikan insight yang lebih mendalam tentang dinamika sistem yang diamati, memungkinkan kita untuk melihat bagaimana perubahan dalam satu variabel dapat memengaruhi variabel lainnya.

In [1]:
import pandas as pd
import tensorflow as tf

Pada kasus ini, kita akan menggunakan sebuah dataset berisikan pengukuran konsumsi daya listrik di satu rumah dengan tingkat pengambilan sampel satu menit selama hampir dua tahun. Tersedia berbagai besaran listrik dan beberapa nilai subpengukuran. Anda dapat mengunduh data pada tautan berikut: [Household Electric Power](https://www.kaggle.com/datasets/uciml/electric-power-consumption-data-set) atau langsung mengakses melalui Google Drive yang sudah disediakan menggunakan kode berikut.

In [2]:
df = pd.read_csv('https://drive.google.com/uc?id=1AZRfFoyekqSYpri5183RmJjciRGz_ood', sep=',',
                     infer_datetime_format=True, index_col='datetime', header=0)
df

C:\Users\HP\AppData\Local\Temp\ipykernel_47476\513246612.py:1: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv('https://drive.google.com/uc?id=1AZRfFoyekqSYpri5183RmJjciRGz_ood', sep=',',


,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
datetime,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
2006-12-16 17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2006-12-16 17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
2006-12-16 17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
2006-12-16 17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0
...,...,...,...,...,...,...,...
2007-02-14 17:19:00,0.636,0.140,241.16,2.6,0.0,0.0,0.0
2007-02-14 17:20:00,0.552,0.000,240.46,2.2,0.0,0.0,0.0
2007-02-14 17:21:00,0.538,0.000,239.74,2.2,0.0,0.0,0.0


Pada studi kasus ini, mari kita buat sebuah fungsi untuk melakukan normalisasi menggunakan fungsi berikut.

Mengapa kita menggunakan sebuah fungsi? Karena kasus ini merupakan multivariate time series di mana data yang kita gunakan memiliki lebih dari satu fitur, maka dari itu alangkah baiknya membuat sebuah fungsi yang dapat digunakan berulang kali. 

In [3]:
def normalize_series(data, min, max):
    data = data - min
    data = data / max
    return data
data = df.values
data = normalize_series(data, data.min(axis=0), data.max(axis=0))

Memastikan jumlah fitur agar tidak keliru untuk menghitung jumlah fitur yang ada mari kita gunakan kode berikut.

In [4]:
N_FEATURES = len(df.columns)

Menentukan proporsi data berdasarkan interval waktu yang sudah ditentukan.

In [5]:
SPLIT_TIME = int(len(data) * 0.5)
x_train = data[:SPLIT_TIME]
x_valid = data[SPLIT_TIME:]

Kode di atas akan memisahkan data menjadi dua bagian yaitu 50% untuk data latih dan 50% sisanya untuk data uji. Seperti yang Anda ingat, jumlah data yang ada pada kasus ini adalah 86.400 baris sehingga dengan menjalankan kode di atas kita akan membagi masing-masing data menjadi 43.200 baris. 



In [6]:
def windowed_dataset(series, batch_size, n_past=24, n_future=24, shift=1):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(size=n_past + n_future, shift=shift, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(n_past + n_future))
    ds = ds.map(lambda w: (w[:n_past], w[n_past:]))
    return ds.batch(batch_size).prefetch(1)

Menentukan parameter untuk membagi data latih dan data uji.

In [7]:
BATCH_SIZE = 32
N_PAST = 24
N_FUTURE = 24
SHIFT = 1
# Kode untuk membuat windowed datasets
train_set = windowed_dataset(series=x_train, batch_size=BATCH_SIZE,
                                 n_past=N_PAST, n_future=N_FUTURE,
                                 shift=SHIFT)
valid_set = windowed_dataset(series=x_valid, batch_size=BATCH_SIZE,
                                 n_past=N_PAST, n_future=N_FUTURE,
                                 shift=SHIFT)

Kode di atas akan mengubah data yang sebelumnya bertipe dataframe menjadi sebuah windowed dataset. Semua parameter yang diperlukan juga sudah ditentukan hingga akhirnya kita sudah memiliki data window latih dan uji yang siap digunakan.



Selanjutnya untuk arsitektur model, gunakan dua buah layer Dense. Perlu Anda perhatikan layer pertama harus memiliki parameter input_shape sesuai dengan ukuran yang sudah kita tentukan sebelumnya yaitu (24, 7). 



In [8]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(64, input_shape=(N_PAST, N_FEATURES)),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(N_FEATURES)
    ])

c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


 myCallback memberikan fleksibilitas untuk mengontrol proses pelatihan model berdasarkan kriteria yang ditetapkan. Dalam kasus ini, pelatihan akan berhenti ketika tingkat kinerja model telah mencapai tingkat yang dianggap memadai, yaitu ketika nilai MAE pada data pelatihan dan data validasi sudah cukup rendah.

In [9]:
class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if (logs.get('mae') < 0.055 and logs.get('val_mae') < 0.055):
                self.model.stop_training = True
 
callbacks = myCallback()

In [10]:
# Kode untuk melakukan menyusun struktur sesuai dengan machine learning
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3) 
model.compile(loss='mae',
                  optimizer= optimizer,
                  metrics=["mae"])

Kode berikut digunakan untuk memulai proses pelatihan model dengan menggunakan dataset pelatihan train_set; mengevaluasi kinerja model pada setiap epoch menggunakan dataset validasi valid_set; dan menggunakan callback hasil modifikasi myCallback untuk menghentikan pelatihan jika kriteria tertentu terpenuhi. 


In [11]:
model.fit(train_set,
          validation_data=(valid_set),
          epochs=100,
          callbacks=callbacks,
          verbose=1
    )

Epoch 1/100
   1344/Unknown 7s 5ms/step - loss: 0.0828 - mae: 0.0828

c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


1349/1349 ━━━━━━━━━━━━━━━━━━━━ 13s 9ms/step - loss: 0.0827 - mae: 0.0827 - val_loss: 0.0575 - val_mae: 0.0575
Epoch 2/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - loss: 0.0601 - mae: 0.0601 - val_loss: 0.0586 - val_mae: 0.0586
Epoch 3/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 12s 9ms/step - loss: 0.0597 - mae: 0.0597 - val_loss: 0.0565 - val_mae: 0.0565
Epoch 4/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 0.0587 - mae: 0.0587 - val_loss: 0.0561 - val_mae: 0.0561
Epoch 5/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 0.0578 - mae: 0.0578 - val_loss: 0.0585 - val_mae: 0.0585
Epoch 6/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 0.0579 - mae: 0.0579 - val_loss: 0.0564 - val_mae: 0.0564
Epoch 7/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 0.0574 - mae: 0.0574 - val_loss: 0.0568 - val_mae: 0.0568
Epoch 8/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 0.0575 - mae: 0.0575 - val_loss: 0.0566 - val_mae: 0.0566
Epoch 9/100
1349/1349 ━━━━━━━━━━━━━━━━━━

Setelah model berhasil dilatih, sekarang Anda sudah bisa melakukan prediksi dengan menggunakan kode berikut.



In [13]:
train_pred = model.predict(train_set)
train_pred[0][0]

1349/1349 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step


array([ 0.39115024,  0.23268387,  0.030233  ,  0.3779718 , -0.00893612,
        0.00388482,  0.85343486], dtype=float32)